In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

from tqdm import tqdm

In [2]:
poly = gpd.read_file('Tuplah/tuplah_5206.geojson', crs='epsg:3857')
tag = gpd.read_file('Tuplah/geotagging_5206.geojson', crs='epsg:3857')
bridge = pd.read_csv('03. Bahan Praktik _ Tools Pemrosesan/07.a Bahan Praktik Pengolahan Tutupan Lahan (Materi Bab 6)/input/kode_bridging_tuplah.csv', sep=';')

In [3]:
poly = poly.dropna(subset='class_id').reset_index(names='poly_id').copy()
poly['class_id'] = poly['class_id'].astype(str)
poly['class_id'] = poly['class_id'].str.split('.').str[0]

In [4]:
bridge['class_id'] = bridge['class_id'].astype(str)
bridge['kode_tuplah'] = bridge['kode_tuplah'].astype(str)

In [5]:
join1 = gpd.sjoin(poly, tag, how='left', predicate='intersects')

In [6]:
mode = join1.groupby('poly_id')['kode_landmark_tipe'].apply(pd.Series.mode).reset_index(level=1, drop=True)
mode_id = mode[mode.groupby(level=0).transform('count') == 1].reset_index()

In [7]:
join2 = poly.merge(mode_id, how='left', on='poly_id')

In [8]:
join3 = pd.merge(join2, bridge[['class_id', 'kode_tuplah']], how='left', left_on='kode_landmark_tipe', right_on='kode_tuplah')

In [9]:
join3.loc[join3['kode_landmark_tipe'].isnull()==False, 'class_id_x'] = join3.loc[join3['kode_landmark_tipe'].isnull()==False, 'class_id_y']
result = join3[['fid', 'id_polygon', 'class_id_x', 'idkab', 'layer', 'path', 'geometry']].copy()
result.rename(columns={
  'fid_left': 'fid',
  'class_id_x': 'class_id'
}, inplace=True)

In [40]:
result.to_file('Tuplah/final.geojson')